# Baseline: popular models

In [ ]:
import requests
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy.stats import mode
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

import random
import os
from rectools import Columns
from rectools.dataset import Dataset

In [ ]:
!pip install rectools==0.2.0

# Get KION dataset 

<a href="https://ods.ai/competitions/competition-recsys-21/data"> Dataset description [ru] </a>


In [ ]:
# download dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [ ]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [ ]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

In [ ]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

## interactions

In [ ]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [ ]:
print(f"Interactions dataframe shape{interactions.shape}") # Количество взаимодействий
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}") # Количество уникальных пользователей в interactions
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}") # Количество уникальных фильмов в interactions

Interactions dataframe shape(5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [ ]:
max_date = interactions['datetime'].max() # Последняя дата взаимодействия
min_date = interactions['datetime'].min() # Первая дата взаимодействия

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


## users

In [ ]:
pd.concat([users.head(), users.tail()])
'''
Датасет по пользователям.
Имеет колонки: id пользователя, возрастная категория, income?, пол, флаг - ребенок или нет
'''

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [ ]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}") # Количество уникальных пользователей в датасете users

Users dataframe shape (840197, 5)
Unique users: 840_197


## items

In [ ]:
pd.concat([items.head(3), items.tail(3)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы...","Гоша, 2019, Россия"


In [ ]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}") # Количество уникальных фильмов

Items dataframe shape (15963, 14)
Unique item_id: 15_963


In [ ]:
def seed_everything(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Prepare data (`dataset` object for rectools)

## Construct item features (`release_year` and `genre`)

 **1. binarize years in release_year feature**

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html

In [ ]:
_, bins = pd.qcut(items["release_year"], 10, retbins=True)
labels = bins[:-1]

year_feature = pd.DataFrame(
    {
        "id": items["item_id"],
        "value": pd.cut(items["release_year"], bins=bins, labels=bins[:-1]),
        "feature": "release_year",
    }
)
year_feature.head()

,id,value,feature
0,10711,1983.0,release_year
1,2508,2012.0,release_year
2,10716,2009.0,release_year
3,7868,2014.0,release_year
4,16268,1897.0,release_year


**2. split genres into list**

In [ ]:
items["genre"] = items["genres"].str.split(",")
items[["genre", "genres"]].head(3)

,genre,genres
0,"[драмы, зарубежные, детективы, мелодрамы]","драмы, зарубежные, детективы, мелодрамы"
1,"[зарубежные, приключения, комедии]","зарубежные, приключения, комедии"
2,"[криминал, зарубежные, триллеры, боевики, ...","криминал, зарубежные, триллеры, боевики, комедии"


In [ ]:
genre_feature = items[["item_id", "genre"]]

In [ ]:
genre_feature = items[["item_id", "genre"]].explode("genre") # Для каждого фильма отдельной строкой записываем каждый жанр
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


**3. Добавим страну как признак**

In [ ]:
items['country'] = items['countries'].str.split(",")
items[["country", "countries"]].sample(3) 

,country,countries
14218,[СССР],СССР
6428,[Россия],Россия
15748,"[Германия, США]","Германия, США"


In [ ]:
country_feature = items[["item_id", "country"]].explode("country")

In [ ]:
country_feature.columns = ["id", "value"]

In [ ]:
country_feature["feature"] = "country"

**4. Добавим актеров как признак**

In [ ]:
items["actor"] = items["actors"].str.split(",")
items[["actor", "actors"]].head(3)

,actor,actors
0,"[Адольфо Фернандес, Ана Фернандес, Дарио Гра...","Адольфо Фернандес, Ана Фернандес, Дарио Гранди..."
1,"[Адам Палли, Брайан Хаски, Дж.Б. Смув, Джей...","Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ..."
2,"[Адриан Холмс, Даррен Шалави, Джерри Вассерм...","Адриан Холмс, Даррен Шалави, Джерри Вассерман,..."


In [ ]:
actor_feature = items[["item_id", "actor"]].explode("actor") # Для каждого фильма отдельной строкой записываем каждого актера
actor_feature.columns = ["id", "value"]
actor_feature["feature"] = "actor"
actor_feature.head()

,id,value,feature
0,10711,Адольфо Фернандес,actor
0,10711,Ана Фернандес,actor
0,10711,Дарио Грандинетти,actor
0,10711,Джеральдин Чаплин,actor
0,10711,Елена Анайя,actor


In [ ]:
item_feat = pd.concat([genre_feature, year_feature, actor_feature])
item_feat = item_feat[item_feat['id'].isin(interactions['item_id'])] # Берем только те id, которые в датасете interactions

In [ ]:
item_feat.sample(10)

,id,value,feature
3621,9930,Дмитрий Шевченко,actor
15069,5257,Джульетт Крух,actor
802,161,Мэттью Уиллиг,actor
15490,5319,NaN,actor
10254,11209,Рональд Рейган,actor
367,1457,Сами Какиашвили,actor
9206,2077,фантастика,genre
5511,9456,Крис Дж.,actor
11020,11724,Лори Петти,actor
12145,7465,для самых маленьких,genre


In [ ]:
item_feat.sort_values('id').head(10)

,id,value,feature
13316,0,русские,genre
13316,0,Антон Виноградов,actor
13316,0,развитие,genre
13316,0,Вадим Бочанов,actor
13316,0,2003.0,release_year
13316,0,для детей,genre
13316,0,сериалы,genre
13316,0,хочу всё знать,genre
13316,0,русские мультфильмы,genre
1296,1,драмы,genre


In [ ]:
item_feat.shape

(55676, 3)

## Construct `User features`

In [ ]:
users.head() # Есть возраст, пол (самое интересное)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


Попробуем возраст

In [ ]:
age_feature = users[["user_id", "age"]]

In [ ]:
age_feature.age.fillna(age_feature.age.mode().item(), inplace=True) # Заполним NaN модой

In [ ]:
age_feature

,user_id,age
0,973171,age_25_34
1,962099,age_18_24
2,1047345,age_45_54
3,721985,age_45_54
4,704055,age_35_44
...,...,...
840192,339025,age_65_inf
840193,983617,age_18_24
840194,251008,age_25_34
840195,590706,age_25_34


In [ ]:
age_feature.age.isnull().sum()

14095

In [ ]:
age_feature.columns = ["id", "value"]

In [ ]:
age_feature["feature"] = "age"

In [ ]:
age_feature = age_feature[age_feature['id'].isin(interactions['user_id'])]

In [ ]:
age_feature

,id,value,feature
0,973171,age_25_34,age
1,962099,age_18_24,age
3,721985,age_45_54,age
4,704055,age_35_44,age
5,1037719,age_45_54,age
...,...,...,...
840189,191349,age_45_54,age
840190,393868,age_25_34,age
840192,339025,age_65_inf,age
840194,251008,age_25_34,age


## Construct `dataset` object to fit models

In [ ]:
dataset = Dataset.construct(
    interactions_df=interactions
    user_features_df=age_feature,
    item_features_df=item_feat,
    cat_item_features=['genre', 'release_year', 'actor']
    cat_user_features = ['age']
)

In [ ]:
begin = interactions["datetime"].max().normalize() - pd.DateOffset(months=1) # Возьмем популярное за месяц

# 1) `Simple popular` by number of interactions

In [ ]:
from rectools.models.popular import PopularModel 

pop = PopularModel(popularity='n_users', begin_from=begin)
pop.fit(dataset);

In [ ]:
reco = pop.recommend(
    users=dataset.user_id_map.external_ids[:1], # В данном случае рекомендуем только пользователю с user_id = 0
    dataset=dataset, # Из какого датасета брать фильмы
    k=10, # Количество рекомендаций
    filter_viewed=False  # True - throw away some items for each user
)
reco.item_id.tolist()

[10440, 15297, 9728, 13865, 3734, 12192, 4151, 11863, 7793, 7829]